In [118]:
import pickle
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.preprocessing import StandardScaler
from datetime import date, datetime, timedelta

In [119]:
# Connect to the Laravel MySQL database
user = 'root'
password = ''
host = 'localhost'
database = 'sales_predictor'

# Create a SQLAlchemy engine
conn = create_engine(f'mysql+pymysql://{user}:{password}@{host}/{database}')

In [120]:
today = datetime.today().day

if today == 7:
    users = pd.read_sql('SELECT * from users', conn)

    for index in users.index:
        user_id = users.loc[index]['id']
        products = pd.read_sql("SELECT * from products where user_id='"+ str(user_id) + "'", conn)
        invoice_products = pd.read_sql("SELECT i.date AS date, ip.product_id AS product_id, ip.quantity AS unit FROM invoice_products ip JOIN invoices i ON ip.invoice_id = i.id where ip.user_id='"+ str(user_id) + "'", conn)
        
        if len(invoice_products) > 0:
            invoice_products['date'] = pd.to_datetime(invoice_products['date'], format='%Y-%m-%d %I:%M:%S %p', errors='coerce')
            
            # Create a pivot table with 'date' as rows and 'product_id' as columns
            pivot_table = invoice_products.pivot_table(index='date', columns='product_id', values='unit', aggfunc='sum', fill_value=0)

            # Reindex to include all dates from the first to the last date in the data
            date_range = pd.date_range(start=invoice_products['date'].min(), end=invoice_products['date'].max())
            pivot_table = pivot_table.reindex(date_range, fill_value=0)

            # Reset index to make 'date' a column again
            pivot_table.index.name = 'date'
            pivot_table = pivot_table.reset_index()

            # Print the pivot table
            print(pivot_table)
        

        



product_id                date 10 11 15   19   21   24 25 27 28   30 31   34  \
0          2022-04-02 16:24:18  0  0  0    0    0  1.0  0  0  0    0  0    0   
1          2022-04-03 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
2          2022-04-04 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
3          2022-04-05 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
4          2022-04-06 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
5          2022-04-07 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
6          2022-04-08 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
7          2022-04-09 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
8          2022-04-10 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
9          2022-04-11 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
10         2022-04-12 16:24:18  0  0  0    0    0    0  0  0  0    0  0    0   
11         2022-04-13 16:24:18  0  0  0 

In [121]:
# # Opening saved model
# with open("model/classifier.pkl", "rb") as file:
#     classifier = pickle.load(file)

# product_data = pd.read_csv("dataset/Project Dataset/product.csv")

# product_data = product_data.sort_values("product").reset_index().drop(columns=['index'])

# today = date.today()
# prediction_dates = [today + timedelta(days = i) for i in range(30)]

# sql = "INSERT INTO `predictions` (`id`, `date`, `product_id`, `unit`, `user_id`, `created_at`, `updated_at`) VALUES "

# for date in prediction_dates:
#     result = classifier.predict(pd.DataFrame({'day_of_year': date.timetuple().tm_yday, 'month':date.month, 'day_of_week':date.weekday()}, index = [0]))
#     i = 0
#     for value in result[0]:
#         if not value == 0:
#             sql += "(NULL, '" + str(date) + "', '" + str(product_data['id'][i]) + "', '" + str(value) + "', '5', CURRENT_TIMESTAMP, CURRENT_TIMESTAMP),"
#         i += 1
# print(sql)